In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
import json

with open('funbigfile.json') as f:
    data = json.load(f)

df = pd.json_normalize(data['bruh'])
df = df.drop(['type', 'uri', 'track_href', 'analysis_url','time_signature', 'duration_ms'], axis=1)
#print(df)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('genre', axis=1), df['genre'], test_size=0.2, random_state=42)

# Encode the target variable
lb = LabelBinarizer()
y_train_encoded = lb.fit_transform(y_train)
y_test_encoded = lb.transform(y_test)

# Define the features to be used for training
features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'valence']
#features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'key', 'liveness', 'speechiness', 'tempo', 'valence']

# Create the feature matrix
X_train = X_train[features]
X_test = X_test[features]

# Train a random forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train_encoded)

#debugging stuff:
print("Testing dataset:")
#lowkey don't need this print(X_test)
print(y_test)
#more debugging stuff 
y_pred = rf.predict(X_test[features])
print("Predicted genres:")
print(lb.inverse_transform(y_pred))


# Evaluate the performance of the model on the test data
accuracy = rf.score(X_test, y_test_encoded)
print("Accuracy:", accuracy)

#with open('smallfile.json') as g:
#    new_data = json.load(g)

#dff = pd.json_normalize(new_data['bruh'])  # Load new JSON data
#dff = dff.drop(['type', 'uri', 'track_href', 'analysis_url','time_signature', 'duration_ms'], axis=1)

Testing dataset:
108     Pop
67     Rock
31     Rock
119     Pop
42     Rock
12      Rap
81     Rock
69     Rock
104     Pop
109     Pop
113     Pop
56     Rock
66     Rock
19      Rap
132     Pop
78     Rock
11      Rap
27     Rock
127     Pop
110     Pop
36     Rock
118     Pop
60     Rock
4       Rap
131     Pop
26     Rock
138     Pop
96     Rock
Name: genre, dtype: object
Predicted genres:
['Pop' 'Rock' 'Rock' 'Pop' 'Rock' 'Rap' 'Rock' 'Rock' 'Pop' 'Pop' 'Rock'
 'Rock' 'Rock' 'Rap' 'Pop' 'Rock' 'Rap' 'Rock' 'Pop' 'Rock' 'Pop' 'Rock'
 'Rock' 'Rap' 'Rap' 'Rock' 'Pop' 'Rock']
Accuracy: 0.6428571428571429
